In [ ]:
#!/usr/bin/env python

import itertools

import numpy
import pandas
import pickle
import random
import scipy.stats

from matplotlib import pyplot

%matplotlib inline

fs1 = 16
fs2 = 14
out_dir = 'plots/'

In [ ]:
in_dir_all = '../01_fitness/'
mutpr_list, mutsr_list, allpr_list, allsr_list = [], [], [], []
for level in numpy.arange(16):
    in_dir = f'{in_dir_all}nlevel_{level:02d}/'
    mut_effect_dict = pickle.load(open(f'{in_dir}mut_effect_dict.pkl', 'rb'))
    mut_all_list = list(mut_effect_dict.keys())
    gt_mut_list = pickle.load(open(f'{in_dir}gt_mut_list.pkl', 'rb'))
    gt_logfit_list = pickle.load(open(f'{in_dir}gt_logfit_list.pkl', 'rb'))
    pr_addlist, sr_addlist = [], []
    backgrounds_all, add_effects_all = [], []
    mut_unidirection = set([])
    random.seed(1075)
    random.shuffle(mut_all_list)
    for mut in mut_all_list:
        mut_effect_list = mut_effect_dict[mut]
        backgrounds = [x[2] for x in mut_effect_list]
        add_effects = [x[3] - x[2] for x in mut_effect_list] 
        pr_addlist.append(scipy.stats.pearsonr(backgrounds, add_effects)[0])
        sr_addlist.append(scipy.stats.spearmanr(backgrounds, add_effects)[0])
        if not (mut[0], mut[2], mut[1]) in mut_unidirection:
            backgrounds_all.extend(backgrounds)
            add_effects_all.extend(add_effects)
            mut_unidirection.add(mut)
    mutpr_list.append(numpy.array(pr_addlist))
    mutsr_list.append(numpy.array(sr_addlist))
    
    backgrounds = numpy.array(backgrounds_all)
    mul_effects = numpy.array(add_effects_all)
    xs = numpy.arange(len(backgrounds), dtype='int')
    pr = scipy.stats.pearsonr(backgrounds[xs], mul_effects[xs])
    sr = scipy.stats.spearmanr(backgrounds[xs], mul_effects[xs])
    allpr_list.append(pr)
    allsr_list.append(sr)
    print(level)

In [ ]:
level_list = numpy.arange(16)
tmp = open('../02_idiosyncracy/sigma_iid.list').read().split()
iid_dict = dict(zip(tmp[::2], tmp[1::2]))
fig, ax = pyplot.subplots(1, 1, figsize=(10, 5))
props = {'color':(0,0,0,0.7), 'linewidth':1.5}
flierprops = {'markeredgecolor':'dimgrey', 'marker':'.', 'markerfacecolor':'dimgrey'}
ax.boxplot(mutpr_list, showfliers=True, medianprops=props, boxprops=props, 
           capprops=props, whiskerprops=props, flierprops=flierprops);
ax.plot(numpy.arange(level_list.shape[0])+1, [x[0] for x in allpr_list],
        'D', markerfacecolor=(0,0,0,0), markeredgecolor='forestgreen', markeredgewidth=2)
# ax.set_xticklabels([f'{x+1:d}\n' + iid_dict[f'{x:d}'] + ''  for x in level_list], horizontalalignment='center')
ax.set_xlabel(r'Highest order of interaction in $n$-order landscapes', fontsize=fs1)
ax.set_ylabel(f"Pearson's $r$ between mutational effect\nand background fitness", fontsize=fs1)
[x.set_fontsize(fs2) for x in ax.xaxis.get_ticklabels()]
[x.set_fontsize(fs2) for x in ax.yaxis.get_ticklabels()]
ax.tick_params(axis='x', rotation=0);
axt = ax.twinx()
axt.plot(numpy.arange(level_list.shape[0])+1, [float(iid_dict[f'{x:d}']) for x in level_list],
        'o', markerfacecolor=(0,0,0,0), markeredgecolor='orangered', markeredgewidth=2)
axt.set_ylim(-0.05, 1.02)
[x.set_fontsize(fs2) for x in axt.yaxis.get_ticklabels()]
[x.set_color('orangered') for x in axt.yaxis.get_ticklabels()]
axt.set_ylabel(r'$I_{id}$', fontsize=fs1, rotation=0, labelpad=15, color='orangered')
fig.tight_layout()
fig.savefig(f'{out_dir}boxplot_pcorr_muts.pdf', dpi=300);